In [ ]:
!pip install roboflow supervision ultralytics torch super_gradients
!pip install -q gdown

import torch

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
SOURCE_VIDEO_PATH = f"{HOME}/<replace_with_your_path.mp4>"
TARGET_VIDEO_PATH = f"{HOME}/<replace_with_your_path.mp4>"
MODEL_NAME = "yolov11"
ULTRALYTICS_MODEL_NAME = "yolo11x"
ROBOFLOW_API_KEY = "<replace_with_your_api_key>"
ROBOFLOW_WORKSPACE = "<replace_with_you_workspace>"
ROBOFLOW_PROJECT_NAME = "perfcam"
ROBOFLOW_PROJECT_NAME_CAPITAL = "PerfCam"
ROBOFLOW_DATASET_VERSION = 3
training_set_version = 3
number_of_classes = 1
ROBOFLOW_LABELS = ['product']
ROBOFLOW_URL = 'https://app.roboflow.com/<your_workspace>/<your_project>/' + str(training_set_version)
file_path = '/content/your-path-' + str(training_set_version) + '/data.yaml'
epocs_count = 200
MODEL = "/content/runs/detect/train/weights/best.pt"

In [ ]:
!nvidia-smi

Sat Dec 28 06:26:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0              40W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.55 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 33.2/235.7 GB disk)


In [ ]:
import supervision as sv
print("supervision.__version__:", sv.__version__)

supervision.__version__: 0.25.1


In [ ]:

from roboflow import Roboflow
!rm -Rf /content/your-path-{training_set_version}/
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(ROBOFLOW_WORKSPACE).project(ROBOFLOW_PROJECT_NAME)
version = project.version(training_set_version)
dataset = version.download(MODEL_NAME)


import yaml

# Create the directory if it doesn't already exist
os.makedirs('/content/your-path-' + str(ROBOFLOW_DATASET_VERSION), exist_ok=True)

# Define the data to be written to the YAML file
data = {
    'names': ROBOFLOW_LABELS,
    'nc': number_of_classes,
    'roboflow': {
        'license': 'Private',
        'project': ROBOFLOW_PROJECT_NAME,
        'url': ROBOFLOW_URL,
        'version': training_set_version,
        'workspace': ROBOFLOW_WORKSPACE
    },
    'test': '../' + ROBOFLOW_PROJECT_NAME_CAPITAL + '-' + str(training_set_version) + '/test/images',
    'train': '../' + ROBOFLOW_PROJECT_NAME_CAPITAL + '-' + str(training_set_version) + '/train/images',
    'val': '../' + ROBOFLOW_PROJECT_NAME_CAPITAL + '-' + str(training_set_version) + '/valid/images'
}


# Write the dictionary to a YAML file
with open(file_path, 'w') as file:
    yaml.dump(data, file)

print(f'File created at {file_path}')

!echo {dataset.location}

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to PerfCam-3 in yolov11:: 100%|██████████| 1035/1035 [00:00<00:00, 5761.95it/s]


File created at /content/<your-path>/data.yaml
/content/PerfCam-3


In [ ]:
!rm -Rf /content/runs

from ultralytics import NAS, YOLO

ULTRALYTICS_MODEL_NAME

'yolo11x'

In [ ]:

# # Load a COCO-pretrained YOLO-NAS-s model
# model = NAS(ULTRALYTICS_MODEL_NAME)

!rm -Rf "/content/{ULTRALYTICS_MODEL_NAME}.pt"

#model = YOLO("/content/pretrained-yolo/" + ULTRALYTICS_MODEL_NAME + '.pt')
model = YOLO(ULTRALYTICS_MODEL_NAME + ".pt")

# # Display model information (optional)
model.info()

#torch.save(model.state_dict(), ULTRALYTICS_MODEL_NAME + '.pt')


100%|██████████| 109M/109M [00:00<00:00, 224MB/s] 


YOLO11x summary: 631 layers, 56,966,176 parameters, 0 gradients, 196.0 GFLOPs


(631, 56966176, 0, 195.9587328)

In [ ]:

# !yolo task=detect \
# mode=train \
# model={ULTRALYTICS_MODEL_NAME}.pt \
# data={file_path} \
# epochs={epocs_count} \
# imgsz=640
checkpoint = torch.load('yolo11x.pt')
print(checkpoint.keys())
#!echo yolo train model=/content/{ULTRALYTICS_MODEL_NAME}.pt data={file_path} epochs={epocs_count} imgsz=640
!yolo train model=/content/{ULTRALYTICS_MODEL_NAME}.pt data={file_path} epochs={epocs_count} imgsz=640
#!yolo train model=/content/yolo11x.pt data=/content/<your-path>/data.yaml epochs=200 imgsz=640


dict_keys(['date', 'version', 'license', 'docs', 'epoch', 'best_fitness', 'model', 'ema', 'updates', 'optimizer', 'train_args', 'train_metrics', 'train_results'])
Ultralytics 8.3.55 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=/content/yolo11x.pt, data=/content/<your-path>/data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, a

In [ ]:
!yolo detect val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

Ultralytics 8.3.55 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLO11x summary (fused): 464 layers, 56,828,179 parameters, 0 gradients, 194.4 GFLOPs
val: Scanning /content/PerfCam-3/valid/labels.cache... 32 images, 1 backgrounds, 0 corrupt: 100% 32/32 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 2/2 [00:01<00:00,  1.45it/s]
                   all         32        147       0.94       0.68      0.805      0.506
Speed: 2.2ms preprocess, 6.7ms inference, 0.0ms loss, 23.1ms postprocess per image
Results saved to runs/detect/val
💡 Learn more at https://docs.ultralytics.com/modes/val
